# Automatic Evaluation on a few samples


In [1]:
# cd to "Directory of "./alignment"
%cd ../../
%ls

c:\ai\workspace\PhD\cir3_anonym\cirrr
 Volume in drive C is OS
 Volume Serial Number is BE2B-F904

 Directory of c:\ai\workspace\PhD\cir3_anonym\cirrr

16/06/2024  14:46    <DIR>          .
16/06/2024  02:48    <DIR>          ..
16/06/2024  14:51                80 .env
16/06/2024  05:14             3,369 .gitignore
16/06/2024  14:46    <DIR>          aimw
16/06/2024  05:09    <DIR>          conf
16/06/2024  14:51             1,432 docker-compose.yml
16/06/2024  15:55    <DIR>          re
16/06/2024  02:51                 6 README.md
               4 File(s)          4,887 bytes
               5 Dir(s)  326,002,257,920 bytes free


In [2]:
# !conda activate phd-align-py3-11
import torch

logger.info(torch.cuda.is_available())
logger.info(torch.version.cuda)

True
12.1


In [3]:
sample = {
    "docid": 54251,
    "doc": "There are two types of credit checks. First is the hard pull which is typically done when you apply for a credit line. The lender will hard pull your file and make his/her decision based on that. This affects your score negatively. You might lose few points for one hard inquiry. Second type is soft pull, which is done as a background check. Typically done by credit card companies to send you a pre-approved offer, or renting an apartment etc. This does not affect your score. One thing to keep in mind is a company will not do a hard pull without your permission, where as they can do soft pulls without you even knowing.  Soft inquiries vs hard inquiries",
    "tokenized_size": 140.0,
    "max_seq_len_exceeded": False,
    "cct_saar": {
        "queries_aspects": [
            {
                "question": "What are the two types of credit checks?",
                "answer": "There are two types of credit checks: hard pull and soft pull. A hard pull is typically done when you apply for a credit line, while a soft pull is done as a background check.",
            },
            {
                "question": "How does a hard pull affect your credit score?",
                "answer": "A hard pull can negatively affect your credit score, causing you to lose a few points.",
            },
            {
                "question": "What is the difference between a hard pull and a soft pull?",
                "answer": "A hard pull is a more invasive check that affects your credit score, while a soft pull is a background check that does not affect your score.",
            },
            {
                "question": "Can a company do a hard pull without your permission?",
                "answer": "No, a company cannot do a hard pull without your permission.",
            },
            {
                "question": "What is the purpose of a soft pull?",
                "answer": "A soft pull is typically done as a background check, such as when a credit card company sends you a pre-approved offer or when you're renting an apartment.",
            },
            {
                "question": "What is the main difference between a hard pull and a soft pull?",
                "answer": "The main difference is that a hard pull affects your credit score, while a soft pull does not.",
            },
            {
                "question": "What is the impact of a hard pull on your credit score?",
                "answer": "A hard pull can cause a temporary decrease in your credit score, but the impact is usually minor.",
            },
        ]
    },
}

In [4]:
from aimw.app.utils.json_utils import JSONProcessor

from loguru import logger

In [6]:
r_dir = "./re/data/fiqa/clean/base/exp_001/"
w_dir = ""
jSONProcessor = JSONProcessor(r_dir + "cct_saar_corpus_cln_split_0.json")
data = jSONProcessor.read_json_file()

## SentenceTransformer Using `BAAI/bge-large-en-v1.5`


In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-large-en-v1.5")


def get_average_scores(data):
    mean_of_mean_score_context_vs_queries = 0.0
    mean_of_mean_score_context_vs_queries_joint = 0.0
    mean_of_mean_score_context_vs_answers = 0.0
    mean_of_mean_score_context_vs_answers_joint = 0.0
    mean_of_mean_score_questions_answers = 0.0
    mean_of_mean_score_questions_joint_vs_answers_joint = 0.0
    mean_of_mean_score_questions_joint_and_answers_joint_vs_context = 0.0
    mean_of_mean_score_questions_and_answers_vs_context = 0.0
    eval_data_size = 0

    for i, doc in enumerate(data):  # data:
        logger.info(f"docid: {doc['docid']}")
        if "cct_saar" in doc.keys():
            if "queries_aspects" in doc["cct_saar"].keys():
                if len(doc["cct_saar"]["queries_aspects"]) > 0:
                    # Extract targets
                    eval_data_size = eval_data_size + 1
                    context = doc["doc"]
                    queries = [
                        q["question"] for q in doc["cct_saar"]["queries_aspects"]
                    ]
                    answers = [a["answer"] for a in doc["cct_saar"]["queries_aspects"]]
                    queries_joint = " ".join(queries)
                    answers_joint = " ".join(answers)

                    number_of_pairs = len(answers)
                    # Embed targets
                    context_embedding = model.encode(context, normalize_embeddings=True)
                    queries_embeddings = model.encode(
                        queries, normalize_embeddings=True
                    )
                    queries_joint_embeddings = model.encode(
                        queries_joint, normalize_embeddings=True
                    )
                    answers_embeddings = model.encode(
                        answers, normalize_embeddings=True
                    )
                    answers_joint_embeddings = model.encode(
                        answers_joint, normalize_embeddings=True
                    )

                    # Compute similarity scores

                    ############# context Vs queries #############
                    similarity_context_queries = (
                        context_embedding @ queries_embeddings.T
                    )
                    logger.info(
                        f"similarity_context_queries: {similarity_context_queries}"
                    )
                    mean_score_context_vs_queries = sum(
                        similarity_context_queries
                    ) / len(
                        similarity_context_queries
                    )  # <----------
                    mean_of_mean_score_context_vs_queries = (
                        mean_of_mean_score_context_vs_queries
                        + mean_score_context_vs_queries
                    )

                    ############# context Vs concatenated queries #############
                    similarity_context_queries_joint = (
                        context_embedding @ queries_joint_embeddings.T
                    )
                    logger.info(
                        f"similarity_context_queries_joint: {similarity_context_queries_joint}"
                    )
                    mean_of_mean_score_context_vs_queries_joint = (
                        mean_of_mean_score_context_vs_queries_joint
                        + similarity_context_queries_joint
                    )

                    ############# context Vs answers #############
                    similarity_context_answers = (
                        context_embedding @ answers_embeddings.T
                    )
                    logger.info(
                        f"similarity_context_answers: {similarity_context_answers}"
                    )
                    mean_score_context_vs_answers = sum(
                        similarity_context_answers
                    ) / len(
                        similarity_context_answers
                    )  # <----------
                    mean_of_mean_score_context_vs_answers = (
                        mean_of_mean_score_context_vs_answers
                        + mean_score_context_vs_answers
                    )
                    ############# context Vs concatenated answers #############
                    similarity_context_answers_joint = (
                        context_embedding @ answers_joint_embeddings.T
                    )
                    logger.info(
                        f"similarity_context_answers_joint: {similarity_context_answers_joint}"
                    )
                    mean_of_mean_score_context_vs_answers_joint = (
                        mean_of_mean_score_context_vs_answers_joint
                        + similarity_context_answers_joint
                    )

                    ################ Questions Vs Answers ################
                    similarity_questions_answers = [
                        x @ y.T for x, y in zip(answers_embeddings, queries_embeddings)
                    ]
                    mean_score_questions_answers = sum(
                        similarity_questions_answers
                    ) / len(
                        similarity_questions_answers
                    )  # <-----------
                    mean_of_mean_score_questions_answers = (
                        mean_of_mean_score_questions_answers
                        + mean_score_questions_answers
                    )
                    logger.info(
                        f"similarty scores: {similarity_questions_answers} | average similarity score: {mean_score_questions_answers}"
                    )

                    ################ Questions Joint Vs Answers Joint ################
                    mean_score_questions_joint_vs_answers_joint = (
                        answers_joint_embeddings @ queries_joint_embeddings.T
                    )  # <----------
                    mean_of_mean_score_questions_joint_vs_answers_joint = (
                        mean_of_mean_score_questions_joint_vs_answers_joint
                        + mean_score_questions_joint_vs_answers_joint
                    )
                    logger.info(
                        f"similarty scores of concatenated Queries Vs Answers: {mean_score_questions_joint_vs_answers_joint}"
                    )

                    ################ (Joint Questions + Joint Answers)/2 Vs Context ################
                    mean_score_questions_joint_and_answers_joint_vs_context = (
                        context_embedding
                        @ ((answers_joint_embeddings + queries_joint_embeddings) / 2).T
                    )  # <----------
                    mean_of_mean_score_questions_joint_and_answers_joint_vs_context = (
                        mean_of_mean_score_questions_joint_and_answers_joint_vs_context
                        + mean_score_questions_joint_and_answers_joint_vs_context
                    )
                    logger.info(
                        f"similarty scores of concatenated Answers embeddings + concatenated Queries embeddings Vs Context: {mean_score_questions_joint_and_answers_joint_vs_context}"
                    )

                    ################ (Questions/n + Answers/n)/2 Answers) Vs Context ################
                    mean_answers_embeddings = sum(answers_embeddings) / len(
                        answers_embeddings
                    )
                    mean_questions_embeddings = sum(queries_embeddings) / len(
                        queries_embeddings
                    )
                    mean_score_questions_and_answers_vs_context = (
                        context_embedding
                        @ ((mean_answers_embeddings + mean_questions_embeddings) / 2).T
                    )  # <----------
                    mean_of_mean_score_questions_and_answers_vs_context = (
                        mean_of_mean_score_questions_and_answers_vs_context
                        + mean_score_questions_and_answers_vs_context
                    )
                    logger.info(
                        f"similarty scores of mean answers,questions embeddings Vs Context: {mean_score_questions_and_answers_vs_context}"
                    )

    return (
        eval_data_size,
        (
            mean_of_mean_score_context_vs_queries / eval_data_size,
            mean_of_mean_score_context_vs_queries_joint / eval_data_size,
            mean_of_mean_score_context_vs_answers / eval_data_size,
            mean_of_mean_score_context_vs_answers_joint / eval_data_size,
            mean_of_mean_score_questions_answers / eval_data_size,
            mean_of_mean_score_questions_joint_vs_answers_joint / eval_data_size,
            mean_of_mean_score_questions_joint_and_answers_joint_vs_context
            / eval_data_size,
            mean_of_mean_score_questions_and_answers_vs_context / eval_data_size,
        ),
    )


# mean_of_mean_score_context_vs_queries = mean_of_mean_score_context_vs_queries / eval_data_size
# mean_of_mean_score_context_vs_queries_joint = mean_of_mean_score_context_vs_queries_joint / eval_data_size
# mean_of_mean_score_context_vs_answers = mean_of_mean_score_context_vs_answers / eval_data_size
# mean_of_mean_score_context_vs_answers_joint = mean_of_mean_score_context_vs_answers_joint / eval_data_size
# mean_of_mean_score_questions_answers = mean_of_mean_score_questions_answers / eval_data_size
# mean_of_mean_score_questions_joint_vs_answers_joint = mean_of_mean_score_questions_joint_vs_answers_joint / eval_data_size
# mean_of_mean_score_questions_joint_and_answers_joint_vs_context = mean_of_mean_score_questions_joint_and_answers_joint_vs_context / eval_data_size
# mean_of_mean_score_questions_and_answers_vs_context = mean_of_mean_score_questions_and_answers_vs_context / eval_data_size

In [7]:
get_average_scores(data)

2024-06-16 16:02:06.838 | INFO     | __main__:get_average_scores:16 - docid: 3
2024-06-16 16:02:07.803 | INFO     | __main__:get_average_scores:40 - similarity_context_queries: [0.52898943 0.68211067 0.58506644 0.53367245 0.5756325  0.76060915
 0.5653718  0.5762904  0.54499257 0.45893148]
2024-06-16 16:02:07.803 | INFO     | __main__:get_average_scores:46 - similarity_context_queries_joint: 0.7120344042778015
2024-06-16 16:02:07.803 | INFO     | __main__:get_average_scores:51 - similarity_context_answers: [0.5467243  0.61753774 0.58939314 0.57899636 0.6150774  0.697689
 0.53532606 0.6085215  0.53775287 0.58703613]
2024-06-16 16:02:07.803 | INFO     | __main__:get_average_scores:56 - similarity_context_answers_joint: 0.7611955404281616
2024-06-16 16:02:07.803 | INFO     | __main__:get_average_scores:63 - similarty scores: [0.89411324, 0.7722802, 0.8500412, 0.85180783, 0.84971434, 0.82390594, 0.8183195, 0.88741446, 0.8092735, 0.6947276] | average similarity score: 0.8251597821712494
2024

(4,
 (0.6708480805835941,
  0.7879372388124466,
  0.6671400761062448,
  0.8094101548194885,
  0.7934548751874403,
  0.8806646764278412,
  0.798673689365387,
  0.6689941138029099))

## BERTScore


In [11]:
import bert_score
from bert_score import score

bert_score.__version__

# hide the loading messages
import logging
import transformers

transformers.tokenization_utils.logger.setLevel(logging.ERROR)
transformers.configuration_utils.logger.setLevel(logging.ERROR)
transformers.modeling_utils.logger.setLevel(logging.ERROR)


def get_average_BERTscores(data):
    mean_P_aj_c, mean_R_aj_c, mean_F1_aj_c = 0.0, 0.0, 0.0
    mean_P_qj_c, mean_R_qj_c, mean_F1_qj_c = 0.0, 0.0, 0.0
    mean_P_qj_aj, mean_R_qj_aj, mean_F1_qj_aj = 0.0, 0.0, 0.0
    eval_data_size = 0
    for i, doc in enumerate(data):  # data:
        logger.info(f"docid: {doc['docid']}")
        if "cct_saar" in doc.keys():
            if "queries_aspects" in doc["cct_saar"].keys():
                if len(doc["cct_saar"]["queries_aspects"]) > 0:
                    # Extract targets
                    eval_data_size = eval_data_size + 1
                    context = doc["doc"]
                    queries = [
                        q["question"] for q in doc["cct_saar"]["queries_aspects"]
                    ]
                    answers = [a["answer"] for a in doc["cct_saar"]["queries_aspects"]]
                    queries_joint = " ".join(queries)
                    answers_joint = " ".join(answers)

                    # Between Answers
                    P_aj_c, R_aj_c, F1_aj_c = score(
                        [answers_joint], [context], lang="en", verbose=True
                    )
                    P_qj_c, R_qj_c, F1_qj_c = score(
                        [queries_joint], [context], lang="en", verbose=True
                    )
                    P_qj_aj, R_qj_aj, F1_qj_aj = score(
                        [queries_joint], [answers_joint], lang="en", verbose=True
                    )

                    mean_P_aj_c = mean_P_aj_c + P_aj_c
                    mean_R_aj_c = mean_R_aj_c + R_aj_c
                    mean_F1_aj_c = mean_F1_aj_c + F1_aj_c
                    mean_P_qj_c = mean_P_qj_c + P_qj_c
                    mean_R_qj_c = mean_R_qj_c + R_qj_c
                    mean_F1_qj_c = mean_F1_qj_c + F1_qj_c
                    mean_P_qj_aj = mean_P_qj_aj + P_qj_aj
                    mean_R_qj_aj = mean_R_qj_aj + R_qj_aj
                    mean_F1_qj_aj = mean_F1_qj_aj + F1_qj_aj

    return (
        eval_data_size,
        (
            mean_P_aj_c / eval_data_size,
            mean_R_aj_c / eval_data_size,
            mean_F1_aj_c / eval_data_size,
        ),
        (
            mean_P_qj_c / eval_data_size,
            mean_R_qj_c / eval_data_size,
            mean_F1_qj_c / eval_data_size,
        ),
        (
            mean_P_qj_aj / eval_data_size,
            mean_R_qj_aj / eval_data_size,
            mean_F1_qj_aj / eval_data_size,
        ),
    )

In [10]:
get_average_BERTscores(data)

2024-06-16 16:04:53.821 | INFO     | __main__:get_average_BERTscores:18 - docid: 3


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.09 seconds, 11.65 sentences/sec


(4,
 (tensor([0.8320]), tensor([0.8679]), tensor([0.8495])),
 (tensor([0.8497]), tensor([0.8631]), tensor([0.8563])),
 (tensor([0.8910]), tensor([0.8522]), tensor([0.8712])))

## Vendi Score for diversity

If diversity between concatned answers and the context is low (close to 1), this means comprehensivness and less hallucination.


In [13]:
from vendi_score import text_utils


def get_average_VENDIscores(data):
    mean_ngram_vs_c_aj = 0.0
    mean_ngram_vs_c_qj = 0.0
    mean_ngram_vs_qj_aj = 0.0
    mean_bert_vs_c_aj = 0.0
    mean_bert_vs_c_qj = 0.0
    mean_bert_vs_qj_aj = 0.0
    mean_simcse_vs_c_aj = 0.0
    mean_simcse_vs_c_qj = 0.0
    mean_simcse_vs_qj_aj = 0.0
    eval_data_size = 0
    for i, doc in enumerate(data):  # data:
        logger.info(f"docid: {doc['docid']}")
        if "cct_saar" in doc.keys():
            if "queries_aspects" in doc["cct_saar"].keys():
                if len(doc["cct_saar"]["queries_aspects"]) > 0:
                    # Extract targets
                    eval_data_size = eval_data_size + 1
                    context = doc["doc"]
                    queries = [
                        q["question"] for q in doc["cct_saar"]["queries_aspects"]
                    ]
                    answers = [a["answer"] for a in doc["cct_saar"]["queries_aspects"]]
                    queries_joint = " ".join(queries)
                    answers_joint = " ".join(answers)

                    c_aj = [context, answers_joint]
                    ngram_vs_c_aj = text_utils.ngram_vendi_score(sents=c_aj, ns=[1, 2])
                    bert_vs_c_aj = text_utils.embedding_vendi_score(
                        sents=c_aj, model_path="bert-base-uncased"
                    )
                    simcse_vs_c_aj = text_utils.embedding_vendi_score(
                        sents=c_aj,
                        model_path="princeton-nlp/unsup-simcse-bert-base-uncased",
                    )

                    c_qj = [context, queries_joint]
                    ngram_vs_c_qj = text_utils.ngram_vendi_score(sents=c_qj, ns=[1, 2])
                    bert_vs_c_qj = text_utils.embedding_vendi_score(
                        sents=c_qj, model_path="bert-base-uncased"
                    )
                    simcse_vs_c_qj = text_utils.embedding_vendi_score(
                        sents=c_qj,
                        model_path="princeton-nlp/unsup-simcse-bert-base-uncased",
                    )

                    qj_aj = [queries_joint, answers_joint]
                    ngram_vs_qj_aj = text_utils.ngram_vendi_score(
                        sents=qj_aj, ns=[1, 2]
                    )
                    bert_vs_qj_aj = text_utils.embedding_vendi_score(
                        sents=qj_aj, model_path="bert-base-uncased"
                    )
                    simcse_vs_qj_aj = text_utils.embedding_vendi_score(
                        sents=qj_aj,
                        model_path="princeton-nlp/unsup-simcse-bert-base-uncased",
                    )

                    mean_ngram_vs_c_aj = mean_ngram_vs_c_aj + ngram_vs_c_aj
                    mean_ngram_vs_c_qj = mean_ngram_vs_c_qj + ngram_vs_c_qj
                    mean_ngram_vs_qj_aj = mean_ngram_vs_qj_aj + ngram_vs_qj_aj
                    mean_bert_vs_c_aj = mean_bert_vs_c_aj + bert_vs_c_aj
                    mean_bert_vs_c_qj = mean_bert_vs_c_qj + bert_vs_c_qj
                    mean_bert_vs_qj_aj = mean_bert_vs_qj_aj + bert_vs_qj_aj
                    mean_simcse_vs_c_aj = mean_simcse_vs_c_aj + simcse_vs_c_aj
                    mean_simcse_vs_c_qj = mean_simcse_vs_c_qj + simcse_vs_c_qj
                    mean_simcse_vs_qj_aj = mean_simcse_vs_qj_aj + simcse_vs_qj_aj

    return (
        eval_data_size,
        (
            mean_ngram_vs_c_aj / eval_data_size,
            mean_ngram_vs_c_qj / eval_data_size,
            mean_ngram_vs_qj_aj / eval_data_size,
        ),
        (
            mean_bert_vs_c_aj / eval_data_size,
            mean_bert_vs_c_qj / eval_data_size,
            mean_bert_vs_qj_aj / eval_data_size,
        ),
        (
            mean_simcse_vs_c_aj / eval_data_size,
            mean_simcse_vs_c_qj / eval_data_size,
            mean_simcse_vs_qj_aj / eval_data_size,
        ),
    )

In [14]:
import warnings

warnings.simplefilter("ignore", UserWarning)
get_average_VENDIscores(data)

2024-06-16 16:06:35.789 | INFO     | __main__:get_average_VENDIscores:15 - docid: 3
2024-06-16 16:06:43.097 | INFO     | __main__:get_average_VENDIscores:15 - docid: 31
2024-06-16 16:06:49.073 | INFO     | __main__:get_average_VENDIscores:15 - docid: 56
2024-06-16 16:06:55.163 | INFO     | __main__:get_average_VENDIscores:15 - docid: 59


(4,
 (1.8516522838990441, 1.9252092854489706, 1.8188529255221262),
 (1.2015637457370758, 1.1242541074752808, 1.1333279013633728),
 (1.5365206003189087, 1.5628820657730103, 1.3434661626815796))

## ROUGE


In [15]:
from rouge import Rouge


def get_average_RougeScores(data):
    mean_rougeL_qj_c = [{"rouge-l": {"r": 0.0, "p": 0.0, "f": 0.0}}]
    mean_rougeL_aj_c = [{"rouge-l": {"r": 0.0, "p": 0.0, "f": 0.0}}]
    mean_rougeL_qj_aj = [{"rouge-l": {"r": 0.0, "p": 0.0, "f": 0.0}}]

    eval_data_size = 0
    rouge_evaluator = Rouge(metrics=["rouge-l"])
    for i, doc in enumerate(data):  # data:
        logger.info(f"docid: {doc['docid']}")
        if "cct_saar" in doc.keys():
            if "queries_aspects" in doc["cct_saar"].keys():
                if len(doc["cct_saar"]["queries_aspects"]) > 0:
                    # Extract targets
                    eval_data_size = eval_data_size + 1
                    context = doc["doc"]
                    queries = [
                        q["question"] for q in doc["cct_saar"]["queries_aspects"]
                    ]
                    answers = [a["answer"] for a in doc["cct_saar"]["queries_aspects"]]
                    queries_joint = " ".join(queries)
                    answers_joint = " ".join(answers)

                    # [{'rouge-l': {'r': 0.2235294117647059, 'p': 0.6129032258064516, 'f': 0.3275862029800833}}]
                    rougeL_qj_c = rouge_evaluator.get_scores(
                        hyps=queries_joint, refs=context
                    )[0]
                    rougeL_aj_c = rouge_evaluator.get_scores(
                        hyps=answers_joint, refs=context
                    )[0]
                    rougeL_qj_aj = rouge_evaluator.get_scores(
                        hyps=queries_joint, refs=answers_joint
                    )[0]

                    mean_rougeL_qj_c[0]["rouge-l"]["r"] = (
                        mean_rougeL_qj_c[0]["rouge-l"]["r"]
                        + rougeL_qj_c["rouge-l"]["r"]
                    )
                    mean_rougeL_qj_c[0]["rouge-l"]["p"] = (
                        mean_rougeL_qj_c[0]["rouge-l"]["p"]
                        + rougeL_qj_c["rouge-l"]["f"]
                    )
                    mean_rougeL_qj_c[0]["rouge-l"]["f"] = (
                        mean_rougeL_qj_c[0]["rouge-l"]["f"]
                        + rougeL_qj_c["rouge-l"]["f"]
                    )

                    mean_rougeL_aj_c[0]["rouge-l"]["r"] = (
                        mean_rougeL_aj_c[0]["rouge-l"]["r"]
                        + rougeL_aj_c["rouge-l"]["r"]
                    )
                    mean_rougeL_aj_c[0]["rouge-l"]["p"] = (
                        mean_rougeL_aj_c[0]["rouge-l"]["p"]
                        + rougeL_aj_c["rouge-l"]["f"]
                    )
                    mean_rougeL_aj_c[0]["rouge-l"]["f"] = (
                        mean_rougeL_aj_c[0]["rouge-l"]["f"]
                        + rougeL_aj_c["rouge-l"]["f"]
                    )

                    mean_rougeL_qj_aj[0]["rouge-l"]["r"] = (
                        mean_rougeL_qj_aj[0]["rouge-l"]["r"]
                        + rougeL_qj_aj["rouge-l"]["r"]
                    )
                    mean_rougeL_qj_aj[0]["rouge-l"]["p"] = (
                        mean_rougeL_qj_aj[0]["rouge-l"]["p"]
                        + rougeL_qj_aj["rouge-l"]["f"]
                    )
                    mean_rougeL_qj_aj[0]["rouge-l"]["f"] = (
                        mean_rougeL_qj_aj[0]["rouge-l"]["f"]
                        + rougeL_qj_aj["rouge-l"]["f"]
                    )

    mean_rougeL_qj_c[0]["rouge-l"]["r"] = (
        mean_rougeL_qj_c[0]["rouge-l"]["r"] / eval_data_size
    )
    mean_rougeL_qj_c[0]["rouge-l"]["p"] = (
        mean_rougeL_qj_c[0]["rouge-l"]["p"] / eval_data_size
    )
    mean_rougeL_qj_c[0]["rouge-l"]["f"] = (
        mean_rougeL_qj_c[0]["rouge-l"]["f"] / eval_data_size
    )

    mean_rougeL_aj_c[0]["rouge-l"]["r"] = (
        mean_rougeL_aj_c[0]["rouge-l"]["r"] / eval_data_size
    )
    mean_rougeL_aj_c[0]["rouge-l"]["p"] = (
        mean_rougeL_aj_c[0]["rouge-l"]["p"] / eval_data_size
    )
    mean_rougeL_aj_c[0]["rouge-l"]["f"] = (
        mean_rougeL_aj_c[0]["rouge-l"]["f"] / eval_data_size
    )

    mean_rougeL_qj_aj[0]["rouge-l"]["r"] = (
        mean_rougeL_qj_aj[0]["rouge-l"]["r"] / eval_data_size
    )
    mean_rougeL_qj_aj[0]["rouge-l"]["p"] = (
        mean_rougeL_qj_aj[0]["rouge-l"]["p"] / eval_data_size
    )
    mean_rougeL_qj_aj[0]["rouge-l"]["f"] = (
        mean_rougeL_qj_aj[0]["rouge-l"]["f"] / eval_data_size
    )

    return (eval_data_size, mean_rougeL_qj_c, mean_rougeL_aj_c, mean_rougeL_qj_aj)

In [16]:
get_average_RougeScores(data)

2024-06-16 16:07:09.142 | INFO     | __main__:get_average_RougeScores:11 - docid: 3
2024-06-16 16:07:09.173 | INFO     | __main__:get_average_RougeScores:11 - docid: 31


2024-06-16 16:07:09.197 | INFO     | __main__:get_average_RougeScores:11 - docid: 56
2024-06-16 16:07:09.219 | INFO     | __main__:get_average_RougeScores:11 - docid: 59


(4,
 [{'rouge-l': {'r': 0.30518501243781093,
    'p': 0.33564351293458833,
    'f': 0.33564351293458833}}],
 [{'rouge-l': {'r': 0.4674945584577115,
    'p': 0.305294510917761,
    'f': 0.305294510917761}}],
 [{'rouge-l': {'r': 0.22650492283431187,
    'p': 0.3251130311122468,
    'f': 0.3251130311122468}}])

## BLEU


In [17]:
from nltk.translate import bleu, bleu_score


def get_average_BlueScores(data):
    mean_bleu_aj_c = 0.0
    mean_bleu_qj_c = 0.0
    mean_bleu_qj_aj = 0.0

    eval_data_size = 0

    for i, doc in enumerate(data):  # data:
        logger.info(f"docid: {doc['docid']}")
        if "cct_saar" in doc.keys():
            if "queries_aspects" in doc["cct_saar"].keys():
                if len(doc["cct_saar"]["queries_aspects"]) > 0:
                    # Extract targets
                    eval_data_size = eval_data_size + 1
                    context = doc["doc"]
                    queries = [
                        q["question"] for q in doc["cct_saar"]["queries_aspects"]
                    ]
                    answers = [a["answer"] for a in doc["cct_saar"]["queries_aspects"]]
                    queries_joint = " ".join(queries)
                    answers_joint = " ".join(answers)

                    bleu_aj_c = bleu(
                        references=[context.split()],
                        hypothesis=answers_joint.split(),
                        weights=(1,),
                    )
                    bleu_qj_c = bleu(
                        references=[context.split()],
                        hypothesis=queries_joint.split(),
                        weights=(1,),
                    )
                    bleu_qj_aj = bleu(
                        references=[answers_joint.split()],
                        hypothesis=queries_joint.split(),
                        weights=(1,),
                    )

                    mean_bleu_aj_c = mean_bleu_aj_c + bleu_aj_c
                    mean_bleu_qj_c = mean_bleu_qj_c + bleu_qj_c
                    mean_bleu_qj_aj = mean_bleu_qj_aj + bleu_qj_aj

    mean_bleu_aj_c = mean_bleu_aj_c / eval_data_size
    mean_bleu_qj_c = mean_bleu_qj_c / eval_data_size
    mean_bleu_qj_aj = mean_bleu_qj_aj / eval_data_size

    return (eval_data_size, (mean_bleu_aj_c, mean_bleu_qj_c, mean_bleu_qj_aj))

In [18]:
get_average_BlueScores(data)

2024-06-16 16:07:51.324 | INFO     | __main__:get_average_BlueScores:11 - docid: 3
2024-06-16 16:07:51.332 | INFO     | __main__:get_average_BlueScores:11 - docid: 31
2024-06-16 16:07:51.334 | INFO     | __main__:get_average_BlueScores:11 - docid: 56
2024-06-16 16:07:51.335 | INFO     | __main__:get_average_BlueScores:11 - docid: 59


(4, (0.14838540377630685, 0.21865774528063278, 0.17381451091717182))

## METEOR


In [21]:
from nltk.translate import meteor
from nltk import word_tokenize


def get_average_MeteorScores(data):

    mean_meteor_q_a = 0.0
    mean_meteor_q_c = 0.0
    mean_meteor_a_c = 0.0
    mean_meteor_aj_c = 0.0
    mean_meteor_qj_c = 0.0
    mean_meteor_qj_aj = 0.0

    eval_data_size = 0

    for i, doc in enumerate(data):  # data:
        logger.info(f"docid: {doc['docid']}")
        if "cct_saar" in doc.keys():
            if "queries_aspects" in doc["cct_saar"].keys():
                if len(doc["cct_saar"]["queries_aspects"]) > 0:
                    # Extract targets
                    eval_data_size = eval_data_size + 1
                    context = doc["doc"]
                    queries = [
                        q["question"] for q in doc["cct_saar"]["queries_aspects"]
                    ]
                    answers = [a["answer"] for a in doc["cct_saar"]["queries_aspects"]]
                    queries_joint = " ".join(queries)
                    answers_joint = " ".join(answers)

                    meteor_aj_c = meteor(
                        references=[word_tokenize(context)],
                        hypothesis=word_tokenize(answers_joint),
                    )
                    meteor_qj_c = meteor(
                        references=[word_tokenize(context)],
                        hypothesis=word_tokenize(queries_joint),
                    )
                    meteor_qj_aj = meteor(
                        references=[word_tokenize(answers_joint)],
                        hypothesis=word_tokenize(queries_joint),
                    )

                    mean_meteor_aj_c = mean_meteor_aj_c + meteor_aj_c
                    mean_meteor_qj_c = mean_meteor_qj_c + meteor_qj_c
                    mean_meteor_qj_aj = mean_meteor_qj_aj + meteor_qj_aj

                    mean_a_q = 0.0
                    mean_c_q = 0.0
                    mean_c_a = 0.0
                    for q, a in zip(queries, answers):
                        a_q = meteor(
                            references=[word_tokenize(q)], hypothesis=word_tokenize(a)
                        )
                        c_q = meteor(
                            references=[word_tokenize(context)],
                            hypothesis=word_tokenize(q),
                        )
                        c_a = meteor(
                            references=[word_tokenize(context)],
                            hypothesis=word_tokenize(a),
                        )
                        mean_a_q = mean_a_q + a_q
                        mean_c_q = mean_c_q + c_q
                        mean_c_a = mean_c_a + c_a

                    mean_a_q = mean_a_q / len(queries)
                    mean_c_q = mean_c_q / len(queries)
                    mean_c_a = mean_c_a / len(queries)

                    mean_meteor_q_a = mean_meteor_q_a + mean_a_q
                    mean_meteor_q_c = mean_meteor_q_c + mean_c_q
                    mean_meteor_a_c = mean_meteor_a_c + mean_c_a

    mean_meteor_q_a = mean_meteor_q_a / eval_data_size
    mean_meteor_q_c = mean_meteor_q_c / eval_data_size
    mean_meteor_a_c = mean_meteor_a_c / eval_data_size

    mean_meteor_aj_c = mean_meteor_aj_c / eval_data_size
    mean_meteor_qj_c = mean_meteor_qj_c / eval_data_size
    mean_meteor_qj_aj = mean_meteor_qj_aj / eval_data_size

    return (
        eval_data_size,
        (mean_meteor_a_c, mean_meteor_q_c, mean_meteor_q_a),
        (mean_meteor_aj_c, mean_meteor_qj_c, mean_meteor_qj_aj),
    )

In [22]:
get_average_MeteorScores(data)

2024-06-16 16:08:31.053 | INFO     | __main__:get_average_MeteorScores:17 - docid: 3
2024-06-16 16:08:32.035 | INFO     | __main__:get_average_MeteorScores:17 - docid: 31
2024-06-16 16:08:32.103 | INFO     | __main__:get_average_MeteorScores:17 - docid: 56
2024-06-16 16:08:32.160 | INFO     | __main__:get_average_MeteorScores:17 - docid: 59


(4,
 (0.12240957033556074, 0.0634707476407487, 0.4848708153735204),
 (0.32550517905570275, 0.2470300445717199, 0.2597391301011615))

## Jaccard Index


In [23]:
def jaccard_similarity(ref, hyp):
    """returns the jaccard similarity between two lists"""
    intersection_cardinality = len(set.intersection(*[set(ref), set(hyp)]))
    union_cardinality = len(set.union(*[set(ref), set(hyp)]))
    return intersection_cardinality / float(union_cardinality)


def get_average_JaccardScores(data):
    mean_aj_c = 0.0
    mean_qj_c = 0.0
    mean_qj_aj = 0.0

    eval_data_size = 0

    for i, doc in enumerate(data):  # data:
        logger.info(f"docid: {doc['docid']}")
        if "cct_saar" in doc.keys():
            if "queries_aspects" in doc["cct_saar"].keys():
                if len(doc["cct_saar"]["queries_aspects"]) > 0:
                    # Extract targets
                    eval_data_size = eval_data_size + 1
                    context = doc["doc"]
                    queries = [
                        q["question"] for q in doc["cct_saar"]["queries_aspects"]
                    ]
                    answers = [a["answer"] for a in doc["cct_saar"]["queries_aspects"]]
                    queries_joint = " ".join(queries)
                    answers_joint = " ".join(answers)

                    aj_c = jaccard_similarity(context, answers_joint)
                    qj_c = jaccard_similarity(context, queries_joint)
                    qj_aj = jaccard_similarity(answers_joint, queries_joint)

                    mean_aj_c = mean_aj_c + aj_c
                    mean_qj_c = mean_qj_c + qj_c
                    mean_qj_aj = mean_qj_aj + qj_aj

    mean_aj_c = mean_aj_c / eval_data_size
    mean_qj_c = mean_qj_c / eval_data_size
    mean_qj_aj = mean_qj_aj / eval_data_size

    return (eval_data_size, (mean_aj_c, mean_qj_c, mean_qj_aj))

In [24]:
get_average_JaccardScores(data)

2024-06-16 16:08:58.127 | INFO     | __main__:get_average_JaccardScores:15 - docid: 3
2024-06-16 16:08:58.128 | INFO     | __main__:get_average_JaccardScores:15 - docid: 31
2024-06-16 16:08:58.129 | INFO     | __main__:get_average_JaccardScores:15 - docid: 56
2024-06-16 16:08:58.130 | INFO     | __main__:get_average_JaccardScores:15 - docid: 59


(4, (0.7550061418929949, 0.7305450677543701, 0.7669600506195431))

## Distinct-n


In [26]:
from aimw.app.services.eval.distinct_n import (
    distinct_n_sentence_level,
    distinct_n_corpus_level,

)


def jaccard_similarity(ref, hyp):

    """returns the jaccard similarity between two lists"""

    intersection_cardinality = len(set.intersection(*[set(ref), set(hyp)]))

    union_cardinality = len(set.union(*[set(ref), set(hyp)]))

    return intersection_cardinality / float(union_cardinality)




def get_average_DistinctNScores(data):

    mean_a_dist1 = 0.0
    mean_q_dist1 = 0.0
    mean_a_dist2 = 0.0
    mean_q_dist2 = 0.0
    mean_aj_dist1 = 0.0
    mean_qj_dist1 = 0.0
    mean_aj_dist2 = 0.0
    mean_qj_dist2 = 0.0
    eval_data_size = 0

    for i, doc in enumerate(data):  # data:

        logger.info(f"docid: {doc['docid']}")

        if "cct_saar" in doc.keys():

            if "queries_aspects" in doc["cct_saar"].keys():

                if len(doc["cct_saar"]["queries_aspects"]) > 0:

                    # Extract targets

                    eval_data_size = eval_data_size + 1

                    context = doc["doc"]

                    queries = [
                        q["question"] for q in doc["cct_saar"]["queries_aspects"]
                    ]

                    answers = [a["answer"] for a in doc["cct_saar"]["queries_aspects"]]
                    queries_joint = " ".join(queries)
                    answers_joint = " ".join(answers)

                    a_dist1 = distinct_n_corpus_level(answers, 1)
                    q_dist1 = distinct_n_corpus_level(queries, 1)

                    a_dist2 = distinct_n_corpus_level(answers, 2)
                    q_dist2 = distinct_n_corpus_level(queries, 2)

                    aj_dist1 = distinct_n_sentence_level(answers_joint, 1)
                    qj_dist1 = distinct_n_sentence_level(queries_joint, 1)

                    aj_dist2 = distinct_n_sentence_level(answers_joint, 2)
                    qj_dist2 = distinct_n_sentence_level(queries_joint, 2)

                    mean_a_dist1 = mean_a_dist1 + a_dist1
                    mean_q_dist1 = mean_q_dist1 + q_dist1
                    mean_a_dist2 = mean_a_dist2 + a_dist2
                    mean_q_dist2 = mean_q_dist2 + q_dist2
                    mean_aj_dist1 = mean_aj_dist1 + aj_dist1
                    mean_qj_dist1 = mean_qj_dist1 + qj_dist1
                    mean_aj_dist2 = mean_aj_dist2 + aj_dist2
                    mean_qj_dist2 = mean_qj_dist2 + qj_dist2

    mean_a_dist1 = mean_a_dist1 / eval_data_size
    mean_q_dist1 = mean_q_dist1 / eval_data_size
    mean_a_dist2 = mean_a_dist2 / eval_data_size
    mean_q_dist2 = mean_q_dist2 / eval_data_size
    mean_aj_dist1 = mean_aj_dist1 / eval_data_size
    mean_qj_dist1 = mean_qj_dist1 / eval_data_size
    mean_aj_dist2 = mean_aj_dist2 / eval_data_size
    mean_qj_dist2 = mean_qj_dist2 / eval_data_size

    return (
        eval_data_size,
        (
            mean_a_dist1,
            mean_q_dist1,
            mean_a_dist2,
            mean_q_dist2,
            mean_aj_dist1,
            mean_qj_dist1,
            mean_aj_dist2,
            mean_qj_dist2,
        ),

    )

In [36]:
average_DistinctNScores = get_average_DistinctNScores(data)
average_DistinctNScores

2024-06-16 16:20:47.922 | INFO     | __main__:get_average_DistinctNScores:21 - docid: 3
2024-06-16 16:20:47.925 | INFO     | __main__:get_average_DistinctNScores:21 - docid: 31
2024-06-16 16:20:47.929 | INFO     | __main__:get_average_DistinctNScores:21 - docid: 56
2024-06-16 16:20:47.933 | INFO     | __main__:get_average_DistinctNScores:21 - docid: 59


(4,
 (0.18682549955800978,
  0.36802365441429297,
  0.6644687015747955,
  0.8235171374143389,
  0.023633874870904632,
  0.04994832337031325,
  0.1527758809227636,
  0.23300355527246447))

## G-Eval / DeepEval - Hallucinations


In [28]:
import os
from aimw.app.core.ai_config import get_ai_settings

os.environ["OPENAI_API_KEY"] = (
    get_ai_settings().openai_api_key
)

os.environ["DEEPEVAL_RESULTS_FOLDER"] = "./output/deepeval/hallucination"

# DeepEval Available GPT models: gpt-4o, gpt-4-turbo, gpt-4-turbo-preview, gpt-4-0125-preview, gpt-4-1106-preview, gpt-4, gpt-4-32k, gpt-4-0613, gpt-4-32k-0613, gpt-3.5-turbo-1106, gpt-3.5-turbo, gpt-3.5-turbo-16k, gpt-3.5-turbo-0125
model_name = "gpt-3.5-turbo-0125"

In [29]:
from deepeval import evaluate
from deepeval.metrics import HallucinationMetric
from deepeval.test_case import LLMTestCase


def get_average_HallucinationScores(data):
    mean_score = 0.0

    eval_data_size = 0

    for i, doc in enumerate(data):  # data:
        logger.info(f"docid: {doc['docid']}")
        if "cct_saar" in doc.keys():
            if "queries_aspects" in doc["cct_saar"].keys():
                if len(doc["cct_saar"]["queries_aspects"]) > 0:
                    # Extract targets
                    eval_data_size = eval_data_size + 1
                    context = doc["doc"]
                    queries = [
                        q["question"] for q in doc["cct_saar"]["queries_aspects"]
                    ]
                    answers = [a["answer"] for a in doc["cct_saar"]["queries_aspects"]]
                    queries_joint = " ".join(queries)
                    answers_joint = " ".join(answers)

                    test_case = LLMTestCase(
                        input=queries_joint,
                        actual_output=answers_joint,
                        context=[context],
                    )
                    metric = HallucinationMetric(threshold=0.5, model=model_name)

                    metric.measure(test_case)

                    logger.info(metric.score)
                    logger.info(metric.reason)
                    logger.info

                    mean_score = mean_score + metric.score
                    # or evaluate test cases in bulk
                    # evaluate([test_case], [metric])

    mean_score = mean_score / eval_data_size

    return (eval_data_size, mean_score)

In [30]:
get_average_HallucinationScores(data)

2024-06-16 16:14:05.644 | INFO     | __main__:get_average_HallucinationScores:12 - docid: 3


Output()

2024-06-16 16:14:09.693 | INFO     | __main__:get_average_HallucinationScores:35 - 1.0
2024-06-16 16:14:09.693 | INFO     | __main__:get_average_HallucinationScores:36 - The hallucination score is 1.00 because the actual output fails to address the key context related to the primary responsibility of software companies and the importance of acquiring marketable skills for students, indicating a complete lack of relevance and accuracy in addressing the given information.
2024-06-16 16:14:09.693 | INFO     | __main__:get_average_HallucinationScores:12 - docid: 31


Output()

2024-06-16 16:14:13.657 | INFO     | __main__:get_average_HallucinationScores:35 - 1.0
2024-06-16 16:14:13.657 | INFO     | __main__:get_average_HallucinationScores:36 - The hallucination score is 1.00 because the actual output aligns with the context provided, offering additional information without any contradictions.
2024-06-16 16:14:13.657 | INFO     | __main__:get_average_HallucinationScores:12 - docid: 56


Output()

2024-06-16 16:14:17.606 | INFO     | __main__:get_average_HallucinationScores:35 - 0.0
2024-06-16 16:14:17.607 | INFO     | __main__:get_average_HallucinationScores:36 - The hallucination score is 0.00 because the actual output aligns perfectly with the context provided, indicating there is no discrepancy or false information present.
2024-06-16 16:14:17.608 | INFO     | __main__:get_average_HallucinationScores:12 - docid: 59


Output()

2024-06-16 16:14:21.421 | INFO     | __main__:get_average_HallucinationScores:35 - 0.0
2024-06-16 16:14:21.423 | INFO     | __main__:get_average_HallucinationScores:36 - The score is 0.00 because the actual output perfectly aligns with the context provided, indicating no hallucinations or inaccuracies in the generated text.


(4, 0.5)

## DeepEval Contextual Relevancy


In [31]:
from deepeval import evaluate
from deepeval.metrics import HallucinationMetric, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase


def get_average_ContextualRelevancyScores(data):
    mean_score = 0.0

    eval_data_size = 0

    for i, doc in enumerate(data):  # data:
        logger.info(f"docid: {doc['docid']}")
        if "cct_saar" in doc.keys():
            if "queries_aspects" in doc["cct_saar"].keys():
                if len(doc["cct_saar"]["queries_aspects"]) > 0:
                    # Extract targets
                    eval_data_size = eval_data_size + 1
                    context = doc["doc"]
                    queries = [
                        q["question"] for q in doc["cct_saar"]["queries_aspects"]
                    ]
                    answers = [a["answer"] for a in doc["cct_saar"]["queries_aspects"]]
                    queries_joint = " ".join(queries)
                    answers_joint = " ".join(answers)

                    metric = ContextualRelevancyMetric(
                        threshold=0.7, model=model_name, include_reason=True
                    )
                    test_case = LLMTestCase(
                        input=queries_joint,
                        actual_output=answers_joint,
                        retrieval_context=[context],
                    )

                    metric.measure(test_case)
                    logger.info(metric.score)
                    logger.info(metric.reason)

                    mean_score = mean_score + metric.score
                    # or evaluate test cases in bulk
                    # evaluate([test_case], [metric])

    mean_score = mean_score / eval_data_size

    return (eval_data_size, mean_score)

In [32]:
contextualRelevancyScores = get_average_ContextualRelevancyScores(data)
contextualRelevancyScores

2024-06-16 16:17:29.193 | INFO     | __main__:get_average_ContextualRelevancyScores:12 - docid: 3


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

2024-06-16 16:17:32.898 | INFO     | __main__:get_average_ContextualRelevancyScores:36 - 1.0
2024-06-16 16:17:32.898 | INFO     | __main__:get_average_ContextualRelevancyScores:37 - The score is 1.00 because the input is focused and directly addresses questions related to the responsibilities of software companies, the importance of acquiring marketable skills, and the impact of education on the job market. It maintains a consistent theme throughout, making the retrieval context entirely relevant.
2024-06-16 16:17:32.898 | INFO     | __main__:get_average_ContextualRelevancyScores:12 - docid: 31


Output()

...

2024-06-16 16:17:39.214 | INFO     | __main__:get_average_ContextualRelevancyScores:36 - 1.0
2024-06-16 16:17:39.216 | INFO     | __main__:get_average_ContextualRelevancyScores:37 - The score is 1.00 because the input contains a comprehensive list of questions related to Health FSAs, individual health insurance premiums, 125 cafeteria plans, and regulations, demonstrating a high level of relevance and specificity.
2024-06-16 16:17:39.216 | INFO     | __main__:get_average_ContextualRelevancyScores:12 - docid: 59


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

2024-06-16 16:17:42.353 | INFO     | __main__:get_average_ContextualRelevancyScores:36 - 1.0
2024-06-16 16:17:42.353 | INFO     | __main__:get_average_ContextualRelevancyScores:37 - The score is 1.00 because the input is focused solely on questions related to Samsung's role, dominance, and significance in the flat screen industry, making the retrieval context completely relevant. Great job!


(4, 1.0)

## DeepEval Faithfulness


In [34]:
from deepeval import evaluate
from deepeval.metrics import (
    HallucinationMetric,
    ContextualRelevancyMetric,
    FaithfulnessMetric,
)
from deepeval.test_case import LLMTestCase


def get_average_FaithfulnessMetric(data):
    mean_score = 0.0

    eval_data_size = 0

    for i, doc in enumerate(data):  # data:
        logger.info(f"docid: {doc['docid']}")
        # if doc["docid"] == 313017:
        #     logger.info(doc)
        if "cct_saar" in doc.keys():
            if "queries_aspects" in doc["cct_saar"].keys():
                if len(doc["cct_saar"]["queries_aspects"]) > 0:
                    # Extract targets
                    eval_data_size = eval_data_size + 1
                    context = doc["doc"]
                    queries = [
                        q["question"] for q in doc["cct_saar"]["queries_aspects"]
                    ]
                    answers = [a["answer"] for a in doc["cct_saar"]["queries_aspects"]]
                    queries_joint = " ".join(queries)
                    answers_joint = " ".join(answers)

                    metric = FaithfulnessMetric(
                        threshold=0.7, model=model_name, include_reason=True
                    )
                    test_case = LLMTestCase(
                        input=queries_joint,
                        actual_output=answers_joint,
                        retrieval_context=[context],
                    )

                    metric.measure(test_case)
                    logger.info(metric.score)
                    logger.info(metric.reason)

                    mean_score = mean_score + metric.score
                    # or evaluate test cases in bulk
                    # evaluate([test_case], [metric])

    mean_score = mean_score / eval_data_size

    return (eval_data_size, mean_score)

In [35]:
faithfulnessMetric = get_average_FaithfulnessMetric(data)
faithfulnessMetric

2024-06-16 16:18:24.325 | INFO     | __main__:get_average_FaithfulnessMetric:16 - docid: 3


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

2024-06-16 16:18:34.596 | INFO     | __main__:get_average_FaithfulnessMetric:42 - 0.7777777777777778
2024-06-16 16:18:34.597 | INFO     | __main__:get_average_FaithfulnessMetric:43 - The score is 0.78 because the actual output presents contradictions by claiming that the company does provide on-the-job training, which goes against the information in the retrieval context that states training workers is not the company's job and software companies may not provide on-the-job training.
2024-06-16 16:18:34.598 | INFO     | __main__:get_average_FaithfulnessMetric:16 - docid: 31


Output()

...

2024-06-16 16:18:55.725 | INFO     | __main__:get_average_FaithfulnessMetric:42 - 0.875
2024-06-16 16:18:55.726 | INFO     | __main__:get_average_FaithfulnessMetric:43 - The score is 0.88 because the claim in the actual output contradicts the retrieval context by stating that using a cafeteria plan to pay for individual premiums is allowed under N. 2013-54, when in fact it is effectively prohibited.
2024-06-16 16:18:55.726 | INFO     | __main__:get_average_FaithfulnessMetric:16 - docid: 59


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

2024-06-16 16:19:03.158 | INFO     | __main__:get_average_FaithfulnessMetric:42 - 1.0
2024-06-16 16:19:03.158 | INFO     | __main__:get_average_FaithfulnessMetric:43 - The score is 1.00 because there are no contradictions present, indicating a high level of faithfulness in the actual output to the retrieval context. Great job!


(4, 0.8819444444444444)